## Lab 2
Выполнила Малышева Екатерина.<br>
В качестве данных для обработки используется корпус, составленный в первой лабораторной работе. (Новостной сайт Lenta.ru)

In [39]:
!pip install nltk sklearn wordcloud pymorphy2 stop_words

In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import pymorphy2
from scipy.sparse import *
import stop_words
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
import json 
%matplotlib inline
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Katya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Katya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [192]:
text_data = pd.read_json("catalog.json", lines=True)
text_data

,article_id,title,category,tags,text
0,https://lenta.ru/news/2022/01/19/sosad/,Россиянка назвала главные ошибки при покупке э...,travel/world,Путешествия,Россиянка отдохнула в Египте и назвала худшие ...
1,https://lenta.ru/news/2022/01/18/opyatetivyshki/,Раскрыто опасное влияние вышек 5G на полеты,travel/world,Путешествия,Несколько крупнейших американских авиакомпаний...
2,https://lenta.ru/news/2022/01/18/maid/,Названы самые грязные вещи в номерах отелей,travel/world,Путешествия,"Горничная Эллиана Мадрид (Elliana Madrid), раб..."
3,https://lenta.ru/news/2022/01/18/kakpochemu/,Названа причина экстренной посадки пассажирско...,travel/world,Путешествия,Нарушение системы управления названо предварит...
4,https://lenta.ru/news/2022/01/18/podorozhali/,Стало известно о значительном подорожании загр...,travel/world,Путешествия,Исполнительный директор Ассоциации туроператор...
...,...,...,...,...,...
5076,https://lenta.ru/news/2018/08/29/obideli/,Рэпер 50 Cent обиделся на мем и пожелал его ав...,media/memes,Интернет и СМИ,Рэп-исполнитель 50 Cent пожелал смерти авторам...
5077,https://lenta.ru/news/2018/08/25/edro_meme/,В сети призвали вернуть кокаин депутатам,media/memes,Интернет и СМИ,В сети с воодушевлением отреагировали на новос...
5078,https://lenta.ru/news/2018/08/25/joejoe_rip/,Умерла самая известная на планете капибара,media/memes,Интернет и СМИ,"Скончался капибара ДжоДжо, который благодаря с..."
5079,https://lenta.ru/news/2018/08/25/binoculars/,Меркель сочли вуайеристкой,media/memes,Интернет и СМИ,В соцсетях отреагировали на решение канцлера Ф...


#### 1. Text preprocessing 

В данной части входные данные будут предобработаны, в частности будут удалены знаки препинания, ненужные символы и стоп-слова, а так же корпус будет токенезирван по предложениям и словам. Слова будут приведены в нормальную форму.

In [145]:
def text_preprocess(text_input):
    
    # копируем текст:
    text_processed = text_input[:]
    
    # переводем текст в lowercase:
    text_processed = text_processed.lower()
    
    # удаляем лишние пробелы:
    text_processed = re.sub(r'\s+', ' ', text_processed)
    
    
    # удалим html теги из текста (если такие есть):
    text_processed =  re.sub(r'\<[^>]*\>', '',  text_processed)
    
    # удалим ссылки:
    text_processed = re.sub(r"https?://[^,\s]+,?", "", text_processed)
    
    # токенизирем по предложению
    text_sent_tokenized = sent_tokenize(text_processed)
    
    # удалим специальны символы цифры и пунктуацию
    symbol_to_exclude = u''.join(['№', '«', 'ђ', '°', '±', '‚', 'ћ', '‰', '…', '»', 'ѓ', 'µ', '·', 'ґ', 'њ', 'ї', 'џ', 'є', '‹',
                                '‡', '†', '¶', 'ќ', '€', '“', 'ў', '§', '„', '”', '\ufeff', '’', 'љ', '›', '•', '—', '‘', 
                                '\x7f', '\xad', '¤', '\xa0'])
    
    regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
    regex_dig = re.compile('[%s]' % re.escape(string.digits))
    regex_symb = re.compile('[%s]' % re.escape(symbol_to_exclude))
    
    text_sent_tokenized  = [regex_symb.sub(' ', sent) for sent in text_sent_tokenized]
    text_sent_tokenized  = [regex_dig.sub(' ', sent) for sent in text_sent_tokenized]
    text_sent_tokenized  = [regex_punct.sub(' ', sent) for sent in text_sent_tokenized]
    
    text_sent_tokenized = [re.sub(r'\s+', ' ', sent) for sent in text_sent_tokenized]
    
    # токенизируем по словам
    text_word_tokenized = [word_tokenize(sent) for sent in text_sent_tokenized]
    
    # приведем слова к нормальной форме 
    morph = pymorphy2.MorphAnalyzer()
    
    text_normalized = []
    for sent in text_word_tokenized:
        text_normalized.append([morph.parse(word)[0].normal_form for word in sent])
        
    # удалим стоп-слова 
    for sent in text_normalized:
        for word in sent:
            if word in stop_words.get_stop_words('ru'):
                sent.remove(word)
                #print(word)
    
    
    text_final =' '.join([' '.join(sent) for sent in text_normalized])
    return  text_final

In [195]:
preprocessed_data = text_data.copy()
for i in  tqdm(range(text_data.shape[0]), desc = 'Article preprocessing progress'):
    preprocessed_data['text'][i]= text_preprocess(text_data['text'][i])
    preprocessed_data['tags'][i] = text_preprocess(text_data['tags'][i])
        

Article preprocessing progress: 100%|██████████████████████████████████████████████| 5081/5081 [50:38<00:00,  1.67it/s]


In [196]:
preprocessed_data

,article_id,title,category,tags,text
0,https://lenta.ru/news/2022/01/19/sosad/,Россиянка назвала главные ошибки при покупке э...,travel/world,путешествие,россиянка отдохнуть египет назвать плохой экск...
1,https://lenta.ru/news/2022/01/18/opyatetivyshki/,Раскрыто опасное влияние вышек 5G на полеты,travel/world,путешествие,крупный американский авиакомпания раскрыть опа...
2,https://lenta.ru/news/2022/01/18/maid/,Названы самые грязные вещи в номерах отелей,travel/world,путешествие,горничная эллиана мадрид elliana madrid работа...
3,https://lenta.ru/news/2022/01/18/kakpochemu/,Названа причина экстренной посадки пассажирско...,travel/world,путешествие,нарушение система управление назвать предварит...
4,https://lenta.ru/news/2022/01/18/podorozhali/,Стало известно о значительном подорожании загр...,travel/world,путешествие,исполнительный директор ассоциация туроператор...
...,...,...,...,...,...
5076,https://lenta.ru/news/2018/08/29/obideli/,Рэпер 50 Cent обиделся на мем и пожелал его ав...,media/memes,интернет сми,рэп исполнитель cent пожелать смерть автор мем...
5077,https://lenta.ru/news/2018/08/25/edro_meme/,В сети призвали вернуть кокаин депутатам,media/memes,интернет сми,сеть воодушевление отреагировать новость обнар...
5078,https://lenta.ru/news/2018/08/25/joejoe_rip/,Умерла самая известная на планете капибара,media/memes,интернет сми,скончаться капибара джоджо благодаря свой друж...
5079,https://lenta.ru/news/2018/08/25/binoculars/,Меркель сочли вуайеристкой,media/memes,интернет сми,соцсеть отреагировать решение канцлер фрг анге...


In [197]:
preprocessed_data.to_excel('preprocessed_corpus.xlsx')# сохраняем обработанные тексты

#### 2. Разделение на test/train и векторизация
Для векторизации используем CountVectorizer - Методика "Мешок Слов". Но до этого разделим корпус на обучающую и тестовую  выборки с пропорцией 70/30.

In [208]:
data = preprocessed_data['text']
y = preprocessed_data['tags']

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

In [209]:
vect = CountVectorizer(min_df=10)

X_train_bow = vect.fit_transform(X_train)
X_test_bow = vect.transform(X_test)

#### 3. ML классификаторы


Рассмотрим наиболее популярные классификаторы (https://tproger.ru/translations/scikit-learn-in-python/). Для улучшения метрик параметры будут подбираться с помощью GridSearch.

##### Наивный Байесовский классификатор 


In [244]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf_NB = MultinomialNB()
clf_NB.fit(X_train_bow, y_train)

y_pred_NB = clf.predict(X_test_bow)
print(classification_report(y_test, y_pred_NB))

               precision    recall  f1-score   support

 интернет сми       0.93      0.79      0.85        99
          мир       0.92      0.94      0.93       266
наука техника       0.90      0.95      0.93       351
  путешествие       0.97      0.95      0.96       268
        спорт       0.98      0.98      0.98       282
     ценность       0.97      0.95      0.96       259

     accuracy                           0.94      1525
    macro avg       0.95      0.93      0.94      1525
 weighted avg       0.95      0.94      0.94      1525



###### Логистическая регрессия 

In [256]:
from sklearn.linear_model import LogisticRegression 


params = {
    'C': np.arange(0.1, 20, 1), # регуляризация
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],
}

clf_LR = LogisticRegression(penalty = 'l2')
grid_LR = GridSearchCV(clf_LR , params, cv=2, verbose = 1)

grid_LR.fit(X_train_bow, y_train)
grid_LR.best_params_

Fitting 2 folds for each of 80 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:   46.0s finished


{'C': 7.1, 'penalty': 'l2'}

In [257]:
y_pred_LR= grid_LR.predict(X_test_bow)
print(classification_report(y_pred_LR, y_test))

               precision    recall  f1-score   support

 интернет сми       0.91      0.93      0.92        97
          мир       0.96      0.92      0.94       277
наука техника       0.96      0.97      0.96       346
  путешествие       0.96      0.98      0.97       264
        спорт       1.00      0.98      0.99       286
     ценность       0.97      0.98      0.97       255

     accuracy                           0.96      1525
    macro avg       0.96      0.96      0.96      1525
 weighted avg       0.96      0.96      0.96      1525



Линейная регрессия справилась с классификацией текстов лучше, чем Наивный Байессовский алгоритм,заметны улучшения по всем метрикам.

##### Дерево решений

In [272]:
from sklearn.tree import DecisionTreeClassifier


clf = DecisionTreeClassifier()
clf.fit(X_train_bow, y_train)

y_pred = clf.predict(X_test_bow)
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

 интернет сми       0.74      0.65      0.69        99
          мир       0.79      0.80      0.80       266
наука техника       0.82      0.84      0.83       351
  путешествие       0.91      0.90      0.90       268
        спорт       0.92      0.94      0.93       282
     ценность       0.87      0.87      0.87       259

     accuracy                           0.85      1525
    macro avg       0.84      0.83      0.84      1525
 weighted avg       0.85      0.85      0.85      1525



При дефолтных параметрах результат работы DecisionTree классификатора на порядок ниже, чем у предыдущих алгоритмов.

##### k - ближащих соседей

In [278]:
from sklearn.neighbors import KNeighborsClassifier


params = {
    'n_neighbors': [2, 4, 8, 16],
    'weights': ['distance', 'uniform'],
    'metric': ['minkowski', 'euclidean']
}

clf_KNN = KNeighborsClassifier(n_neighbors=3)


grid_KNN = GridSearchCV(clf_KNN , params, cv=2, verbose = 1)

grid_KNN.fit(X_train_bow, y_train)
grid_KNN.best_params_

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  32 out of  32 | elapsed:    6.8s finished


{'metric': 'minkowski', 'n_neighbors': 2, 'weights': 'distance'}

In [279]:
y_pred_KNN= grid_KNN.predict(X_test_bow)
print(classification_report(y_pred_KNN, y_test))

               precision    recall  f1-score   support

 интернет сми       0.45      0.87      0.60        52
          мир       0.71      0.85      0.77       222
наука техника       0.61      0.97      0.75       220
  путешествие       0.78      0.93      0.85       224
        спорт       1.00      0.45      0.62       623
     ценность       0.71      0.99      0.83       184

     accuracy                           0.73      1525
    macro avg       0.71      0.84      0.73      1525
 weighted avg       0.81      0.73      0.72      1525



При подборе гиперпараметров удалось улучшить accuracy до 0.73, что сильно уступает рассмотренным ранее алгоритмам. Судя по графе support, алгоритм склонен сильно переопределять категорию "спорт" и недооценивать "интернет и сми".

##### Random forest 

In [306]:
from sklearn.ensemble import RandomForestClassifier

clf_RFC = RandomForestClassifier()
clf_RFC.fit(X_train_bow, y_train)
#print(clf.get_params())
y_pred_RFC = clf_RFC.predict(X_test_bow)
print(classification_report(y_test, y_pred_RFC))

               precision    recall  f1-score   support

 интернет сми       0.97      0.72      0.83        99
          мир       0.89      0.94      0.92       266
наука техника       0.92      0.94      0.93       351
  путешествие       0.96      0.95      0.95       268
        спорт       0.98      0.99      0.98       282
     ценность       0.93      0.94      0.94       259

     accuracy                           0.94      1525
    macro avg       0.94      0.91      0.92      1525
 weighted avg       0.94      0.94      0.94      1525



При стандартных параметрах у Random Forest и Наивного Байесовского алгоритма метрики имеют приближенные значения и accuracy достигает достаточно высоких значений - 0.94.

##### Метод опорных векторов

In [242]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train_bow, y_train)

y_pred = clf.predict(X_test_bow)
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

 интернет сми       0.93      0.79      0.85        99
          мир       0.92      0.94      0.93       266
наука техника       0.90      0.95      0.93       351
  путешествие       0.97      0.95      0.96       268
        спорт       0.98      0.98      0.98       282
     ценность       0.97      0.95      0.96       259

     accuracy                           0.94      1525
    macro avg       0.95      0.93      0.94      1525
 weighted avg       0.95      0.94      0.94      1525



In [309]:
params = {
    'C': [0.1, 1, 10], 
    'gamma': [1, 0.1, 0.01], 
    'kernel': ['rbf', 'poly', 'sigmoid']
}

gs_svm = GridSearchCV(
    SVC(random_state=42),
    params,
    verbose=3,
    cv=2,
)


gs_svm.fit(X_train_bow, y_train)
gs_svm.best_params_

Fitting 2 folds for each of 27 candidates, totalling 54 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.231, total=  10.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.5s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.231, total=   7.6s
[CV] C=0.1, gamma=1, kernel=poly .....................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.1s remaining:    0.0s


[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.624, total=   6.7s
[CV] C=0.1, gamma=1, kernel=poly .....................................
[CV] ......... C=0.1, gamma=1, kernel=poly, score=0.667, total=   6.7s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ...... C=0.1, gamma=1, kernel=sigmoid, score=0.380, total=   6.9s
[CV] C=0.1, gamma=1, kernel=sigmoid ..................................
[CV] ...... C=0.1, gamma=1, kernel=sigmoid, score=0.376, total=   7.3s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.231, total=   7.2s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.231, total=  14.1s
[CV] C=0.1, gamma=0.1, kernel=poly ...................................
[CV] ....... C=0.1, gamma=0.1, kernel=poly, score=0.624, total=  12.2s
[CV] C=0.1, gamma=0.1, kernel=poly ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  6.9min finished


{'C': 1, 'gamma': 0.01, 'kernel': 'sigmoid'}

In [311]:
y_pred = gs_svm.predict(X_test_bow)
print(classification_report(y_pred, y_test))

               precision    recall  f1-score   support

 интернет сми       0.92      0.91      0.91       100
          мир       0.92      0.89      0.90       275
наука техника       0.94      0.95      0.94       347
  путешествие       0.94      0.96      0.95       262
        спорт       1.00      0.98      0.99       287
     ценность       0.95      0.97      0.96       254

     accuracy                           0.95      1525
    macro avg       0.94      0.94      0.94      1525
 weighted avg       0.95      0.95      0.95      1525



#### Выводы
Из рассмотренных алгоритмов классификации наиболее высокие метрики показала Логистическая регрессия - accuracy достигла 96%. Более того, все остальные метрики - precision, f1, recall - оставались выше 90%.
Также высокие метрики были у метода опорных векторов (SVC) - accuracy - 95%.
Наиболее худшим алгоритмом для даннрой задачи классификации оказался метод k - ближайших соседей, accuracy достигло все лишь 73%. Более того, алгоритм неравномерно определял классы (имел тендентцию переопределять категорию "спорт").
